# Introduction to Dash

### Learning Objectives:
- [Introduction: What is Dash?](#Introduction:-What-is-Dash?)
- [App Layout: Dash Components](#App-Layout:-Dash-Components)
- [The Callback & Decorators](#The-Callback-&-Decorators)
- [MNIST Image Detector Dashboard](#MNIST-Image-Detector-Dashboard)
- [Bootstrap & CSS](#Bootstrap-&-CSS)

# Introduction: What is Dash?

__Dash__ is a a productive framework made by Plotly for building web applications with custom user interfaces that is available in Python, R and Julia. They are written on top of Flask, Plotly.js and React.js, which enables us to create analytical web applications for data visualization in pure Python without the need of HTML, CSS and JavaScript! Dash apps are rendered in the web browser and can easily be deployed and shared via URLs.

Wondering what these look like? Luckily, Plotly has a gallery of example apps for us to check out [here](https://dash-gallery.plotly.host/Portal/)! Dash apps can be used in a vast amount of fields that require elegant and intuitive UIs to be built around models and data visualization, ranging from finance, medicine, bioinformatics, mining, machine learning, and many more! 

Furthermore, they can be used over plain data dashboards, which are information management tools that visually tracks, analyzes and displays metrics and key data points to monitor the health of a business, department or process. Common dashboards are passive systems that present pre-generated data, whereas dash web apps allow interaction with parameters, datasets, sending emails with actions and items running AI models with a touch a button among other features!

So how do we make a Dash app? These apps are made with four main components that we are going to cover today:
- Initialization of the app: creating a dash.Dash (for a webapp) object or JypyterDash (for a Jupyter notebook) object
- App layout: Structuring the layout of the page with Dash core components and Dash HTML components
- Plotly graphs: Dash uses Plotly graphs data visualization
- Callback: Enables interactivity between user inputs and outputs in our app

Those three components are responsible for 100% of the functionality of (offline) dash apps. We can also create visualize our Dash apps on Jupyter notebooks by using the _jupyter\_dash_ library. Here we see a demo of a Dash app that we can use on Jupyter notebooks available [here](https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e).

In [2]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

# App Layout: Dash Components

Before we dive into __Dash components__, let us take a quick glance of the standard structure of basic webpages. Generally speaking, the basis of any webpages are __HyperText Markup Language (HTML)__, __Cascading Style Sheets (CSS)__, and __JavaScript__. As shown in the diagram:
- HTML is responsible for the content & structure displayed on a webpage via a nested structure of native elements (html, head, body, div, p, h1, etc)
- CSS is responsible for the presentation of the webpage by styling of individual/groups of elements
- JavaScript is responsible for the behaviour of the webpage, enabling interactivity with the user

<img style="display:block; margin-left: auto; margin-right: auto; width: 50%" src=https://3.bp.blogspot.com/-7pXla4gYcSU/XB-aQElY_zI/AAAAAAAAHTA/soP-84BDatwdPyeo3jrwtQxQAvzbQIbUgCLcBGAs/s1600/js_css_html.png>


What Dash provides us is (roughly) an abstraction of these primitive building blocks, enabling us to get all their functionality on a webpage using pure Python. The _app_ object that we created has a ___.layout___ attribute, which we can assign our structure of components. Components in Dash apps come in two flavours: __HTML Components__ and __Dash Core Components__. The general idea behind the layout structure of our app is very similar to how we would structure the HTML for a page, and in fact, is roughly what Dash is doing in the background. Let us take a closer look at the _app.layout_ structure for the demo Dash app we have seen above:

<img style="position:relative; left:300px;" src="images/dash_layout_structure.png">

After a close inspection we can see that:
- __app.layout follows the same nesting structure as ordinary HTML__
    - We see that the primary container (html.Div) has other objects inside it, and the html.Label contains both the text "colorscale" and the dropdown element
- This nesting structure is obtained in Python such that elements that 'contain' other elements (like html.Div) take in a list as their argument, where each value of the list are the constituent components!
    - This means that html.Div contains the title, graph and label; the label contains the description and dropdown menu, and so on
- We can use Dash Core components (dcc) and HTML Components (html) interchangeably in our layout
- We can also pass in _className_ or _id_ for keyword arguments (such as for the dropdown object) as identifiers

And this is how we can generate the layout structure for our Dash app. For those of you that are web-dev savy, it is useful to note that __you can also edit the style attribute of Dash HTML elements__. For those of you that are not that familiar with HTML, don't worry, as you can get away with creating apps that are essentially composed of 95% Dash Core components.

But the whole point of a Dash app is that it's not static, but should instead enable the user to interact with whatever you've built. This is where callbacks come in!

# The Callback & Decorators

To add interactivity, Dash apps use __callback functions:__ Python functions that are automatically called by Dash whenever an input component's property changes. Dash abstracts what can often be quite cumbersome to do with the ___@app.callback__ decorator. In short a __decorator__ is any callable Python object that is used to modify a function or a class. Python provides the '@' as syntax to call decorators above a given function definition to add some extra functionality.

Before diving into Dash app callbacks, let's try and understand what decorators are and how they work. Below we have an example of how decorators are implemented in Python.

In [3]:
# Playing with decorators

# Decorator: takes in any input function, and adds to it the print statements
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print("\U0001F60E")
        print("You've been decorated")
        return original_function(*args, **kwargs)
    return wrapper_function

# Original function
def hello_world():
    print("hello world")

# What decorators do under the hood
hello_world2 = decorator_function(hello_world)

# How we can apply them directly
@decorator_function
def hello_world3():
    print("hello world")

hello_world()
print()
hello_world2()
print()
hello_world3()

hello world

😎
You've been decorated
hello world

😎
You've been decorated
hello world


In [7]:
# Practice: Code a decorator function that adds takes in a function and returns the same function, with added functionality of printing runtime

## Decorator function:
def my_timer(original_function):
    from time import time

    def wrapper(*args, **kwargs):
        t1 = time()
        result = original_function(*args, **kwargs)
        t2 = time() - t1
        print("{} ran in: {} s".format(original_function.__name__, t2))
        return result
    return wrapper

# Example code:
@my_timer
def alphabet():
    print("a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z")
    print("Now, I know my ABC's. Next time won't you sing with me?")

alphabet()

a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z
Now, I know my ABC's. Next time won't you sing with me?
alphabet ran in: 0.0001919269561767578 s


And there you have it, decorators can be quite useful for repetitive functionality that we do not need to include in the original function definitions every time. Now that we gone over decorators, we can understand the @app.callback decorator. Consider the callback for the example we have given above:

<img style="display:block; margin-left: auto; margin-right: auto; width: 50%" src="images/callbacks.png">

While this is a simple example, we can discern a few things already:
- The @app.callback decorator adds the 'callback functionality' to the function we defined
- We always need a function (i.e. _update_figure_) to use the callback decorator
    - Everytime an input is changed, we want one (or more) of our outputs to change correspondingly. This is encoded in the function that we 'decorate'
- The callback decorator takes extra arguments: one list containing __Input__ objects and one list containing __Output__ objects
    - These arguments for our decorator gives information as to what are all the inputs it needs to track in case they change and what are all the outputs that may be edited in our callback function
- The __Input__ and __Output__ objects take in the __ids of the Dash components in app layout__ so that Dash knows what are the inputs and outputs in our app and the type of input/output (figure, children, src, value, etc)

__IMPORTANT:__ It may not be obvious here, but it is important to know that for the function we are decorating __we take in as many arguments as inputs we set for the decorator__, and __return as many outputs as we set for the decorator.__

__And that's pretty much it.__

From these building blocks you can create any (basic) Dash app with a given level of functionality. We will now create our very own Dash app using the vanilla CNN we trained on the MNIST dataset.

# MNIST Image Detector Dashboard

Now we'll let you make your own Dash App (sort of) from scratch! At this point, we have all either trained a CNN on the MNIST dataset or seen a CNN be trained on the MNIST dataset, generally getting a pretty high test accuracy given that it's a pretty straightforward dataset. Here, we provide you with a saved PyTorch cnn that has been trained and achieved ~98% accuracy! Your task is to go through the _mnist_app.py_ file and complete what is missing. If done correctly you will get an interactive Dash App that allows you to drag an drop an MNIST image and will predict the correct digit and display the probability of each digit! It will look something like this:

<img style="display:block; margin-left: auto; margin-right: auto; width: 50%" src="images/workingapp.png">

And with minimal to no styling at all, you will all have your first interactive Dash App! Below are some pointers that you will need to be aware of before you get started:

- If an error does not occur, Dash Apps will automatically update to code changes!
- Make sure you are in the same directory as your .py file before running!
- To make sure we display something before we upload an image, we start with a default image saved in the images directory 'images/default.jpg'
- In the 'images' directory you will also find images of an arrow and of all digits in a seven-segment display format
- The Dash Core Component, dcc.Upload, returns input images as base64-encoded strings. Likewise, the html.Img component doesn't work well for updating locally stored images dynamically unless the address is given as a base64-encoded string. Hence, to save you time:
    - To convert these to NumPy arrays, use the string2img function from utils.py
    - To encode NumPy arrays as base64, use the img2string function from utils.py
- Lastly, once everything works (if time allows), add a small message to denote whether the current image being displayed is the default starting image or not

Please read through the pre-written code in the app file and make sure you understand what everything means! If you can complete this Dash App you can write your own any time.




# Bootstrap & CSS

### CSS

In the code already written, you may have noticed how both Dash Core components and Dash HTML Components have taken in _style_ as a keyword argument, which takes in a dictionary of properties. This is analogous to how inline styles works with ordinary HTML elements, and enables us to tweak specific properties of a given element. For example, 'width':'50%' assigns the width of the element to 50% of the width of the container in which it is in (in this case, generally html.Div). This allowed us to manually position and resize our figures, images, text, etc. Styling is just as important as the structure of the webpage itself!

We can take Inline Styles a step further and write what are known as __Cascading Styling Sheets (CSS)__, which are external files that tell us how we should style different elements in our webpage. In the simplest form of CSS, we can target individual components via their 'id' keyword argument, or a group of components via the 'className' keyword argument, all without clustering our Python file with Inline Styles. By giving specific style properties to all elements of the same class, we can get themed elements (e.g. all buttons have same colour, width, padding, font, etc). While we won't go over it today, keep in mind that CSS can do much more than this!

We will not go over writing your own CSS today, but will rather focus on how to link CSS files to your app. Dash has a simple mechanism for importing css files: __create a directory called 'assets' in the same directory as the app, then place desired CSS files in this 'assets' directory__.

In fact, we already have a CSS file in our assets directory, which is what we used to display the three images in our app side by side! You may find various useful element structures and styes online that you would like to implement in your own apps, and you can easily do so this way.

### Bootstrap

So now that we have CSS, we're set in terms of styling... right? Not quite. It may take a lot of learning and digging to create appropriately styled elements in our Dash App, whether we're using Inline Styles or CSS. Very often, it's a better use of our time using the work already done by amazing people who really know their stuff. One simple way in which we can do this, is by using __Dash Bootstrap Components__. __Bootstrap__ is a popular front-end open source toolkit, and they have their own components library available for Dash! Check out their documentation [here](https://dash-bootstrap-components.opensource.faculty.ai/).

How do we use them? It's very straightforward, we just have to carry out two steps:
- Link a Bootstrap compatible stylesheet to our app. By giving us options of stylesheets, we can choose between various themes and styles on the go!
- Use _dbc_ components just as we have been using _dcc_ and _html_

Let us see how this works on our basic demo first:

In [25]:
import dash_bootstrap_components as dbc
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

KeyboardInterrupt: 

As we saw above, one way to link an external stylesheet (CSS file) to our app was to input '\[dbc.themes.THEME\]' for the _external\_stylesheets_ keyword argument. What is the value of '\[dbc.themes.THEME\]'?

In [26]:
print(dbc.themes.CERULEAN)
print(dbc.themes.CYBORG)
print(dbc.themes.BOOTSTRAP) # default

https://stackpath.bootstrapcdn.com/bootswatch/4.5.0/cerulean/bootstrap.min.css
https://stackpath.bootstrapcdn.com/bootswatch/4.5.0/cyborg/bootstrap.min.css
https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css


Surprise! We get the URL of the CSS file corresponding to the chosen theme! We can achieve the same outcome by downloading the CSS file from the link and putting it in our assets folder. Now that you have seen this, download the CSS file for a theme of your choice, put it in your assets directory and check your MNIST app. 

__NOTE:__ You should only link the stylesheet with the _external\-stylesheets_ command, or by putting your CSS file in the 'assets' directory, __NEVER BOTH__

Now that we have learned how to link external stylesheets, let's add Dash Bootstrap Components. How do we do that? The exact same way as we used html and dcc components! Below we replace the Dash Dropdown Core Component with the Dash Bootstrap Select Component (both provide the same functionality).

In [5]:
import dash_bootstrap_components as dbc
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        # dcc.Dropdown(
        #     id='colorscale-dropdown', clearable=False,
        #     value='plasma', options=[
        #         {'label': c, 'value': c}
        #         for c in px.colors.named_colorscales()
        #     ])
        dbc.Select(
            id='colorscale-dropdown',
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

# Congratulations

### Challenges:
- Complete MNIST Dash app if not done so in class
- Data dashboards are data visualizationt tools used by many companies to understand analytics that matter to a business, department or project. Look into these and what kind of metrics they are used to visualize and create your own interactive data dashboard app for this dataset [here](https://www.kaggle.com/milanzdravkovic/pharma-sales-data)